In [27]:
import time, os
from utils.vector_db import vectorDataBase
from tqdm import tqdm
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PDFMinerLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [ ]:
os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"]="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"
os.environ["LANGSMITH_PROJECT"]="simple_rag_llm"

# **LLM's**

In [2]:
llm = GoogleGenerativeAI(model="gemini-2.5-flash", verbose=True)

In [3]:
print(llm.invoke("Can you teach me bankai??"))

Ah, Bankai! A noble pursuit, indeed! To unleash the final, most powerful release of your Zanpakuto is the dream of every Shinigami.

However, before we delve into the intense training, let's clarify something crucial: **Bankai is a fictional concept from the anime and manga series *Bleach*.** It's not something that can be learned or achieved in the real world.

But since you're asking to be "taught," let's dive deep into the *Bleach* lore and understand exactly what Bankai is, and how one would supposedly achieve it within that universe!

---

## **Teaching You Bankai (The Fictional Way!)**

### **What is Bankai?**

Bankai (卍解, "Full Release" or "Final Release") is the second and final, most powerful form a Zanpakuto can take. It's the ultimate manifestation of a Shinigami's bond with their sword spirit.

*   **Power Boost:** A Bankai typically amplifies a Shinigami's power by 5 to 10 times that of their Shikai.
*   **Unique Abilities:** It often completely changes the appearance and 

# **Vector Stores**

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    task_type="retrieval_document"
)

# db_creator = vectorDataBase()
# pdf_store = db_creator.initalize_empty_db(
#     emdeddingFunction=embeddings
# )

pdf_store = FAISS.load_local(r"C:\Stack overflow\RAG\db")

Previous vector db is over written by this function


# **Loading from websites**

In [ ]:
loader = PDFMinerLoader(
    r"E:\jjk_pdf.pdf"
)
loaded_docs = loader.load()

In [6]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50,
    length_function=len
)

docs = splitter.split_documents(loaded_docs)
print(len(docs))

648


In [7]:
with tqdm(total=len(docs), desc="Adding docs", unit="docs") as pbar:
    for i in range(0, len(docs), 10):
        batches = docs[i:i+10]
        length = len(pdf_store.add_documents(batches))
        time.sleep(30)
        pbar.update(length)

Adding docs: 100%|██████████| 648/648 [34:01<00:00,  3.15s/docs]


In [8]:
retiever = pdf_store.as_retriever(k=5)

In [9]:
retiever.invoke("is sukuna really villan?")

[Document(id='bdf3e075-181c-4669-b92b-809fd0a79306', metadata={'producer': 'Skia/PDF m116', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36', 'creationdate': '2023-09-24T19:45:22+00:00', 'moddate': '2023-09-24T19:45:22+00:00', 'total_pages': 33, 'source': 'E:\\jjk_pdf.pdf'}, page_content='But that all changes when the strongest cursed spirit Ryumen Sukuna is incarnated in the body'),
 Document(id='e9b7e8c8-c0f5-45f9-88d9-b3abbf7dd47f', metadata={'producer': 'Skia/PDF m116', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36', 'creationdate': '2023-09-24T19:45:22+00:00', 'moddate': '2023-09-24T19:45:22+00:00', 'total_pages': 33, 'source': 'E:\\jjk_pdf.pdf'}, page_content='Jujutsu Kaisen'),
 Document(id='901e6347-8a81-49c4-93c9-67e44350eccd', metadata={'producer': 'Skia/PDF m116', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWe

In [10]:
pdf_store.save_local(r"db")

# **Prompt**

In [21]:
template = """

Answer the following question based on given context {context} 

Question: {question}

You are an expert debate moderator and analyst. Using only the information provided in the context, facilitate a structured debate on the topic presented in the question, then determine a clear winner with comprehensive statistics and visual representations.

## Debate Structure:

**Phase 1: Opening Statements**
Based on the context provided, present 3-4 distinct perspectives on the topic. For each perspective:
- State the core position clearly using evidence from the context
- Provide 2-3 strongest supporting arguments found in the context
- Include relevant statistics, data, or expert opinions from the provided context
- Acknowledge the strongest counterargument to this position found in the context

**Phase 2: Cross-Examination**
For each perspective, using only context information:
- Present direct challenges from opposing viewpoints
- Show responses to counterarguments
- Provide additional evidence from context to strengthen positions
- Identify potential weaknesses or limitations mentioned in the context

**Phase 3: Rebuttal Round**
- Present final strengthened arguments for each perspective using context
- Address remaining critical points found in the provided information
- Summarize why each perspective should win based on context evidence

**Phase 4: Comprehensive Analysis & Winner Declaration**

Evaluate each perspective using these weighted criteria based solely on context information:
1. **Logical Consistency** (25%): Internal coherence based on context evidence
2. **Evidence Quality** (25%): Strength of supporting data from context
3. **Argument Strength** (20%): Persuasiveness based on context information
4. **Counterargument Handling** (15%): How well opposing views are addressed in context
5. **Real-World Applicability** (10%): Practical implications mentioned in context
6. **Ethical Considerations** (5%): Moral implications found in context

## Final Winner Declaration:

**WINNER: [Perspective Name]**

**Victory Margin**: [Decisive/Strong/Narrow] - [Overall Score X.X/10]

**Detailed Scoring Breakdown**:
- Perspective 1: [Score/10] - [Justification from context]
- Perspective 2: [Score/10] - [Justification from context]  
- Perspective 3: [Score/10] - [Justification from context]
- [Additional perspectives as needed]

**Key Victory Factors** (based on context):
1. [Primary reason supported by context evidence]
2. [Secondary reason from context]
3. [Additional factors from context]

**Statistical Summary** (derived from context):
- Total arguments presented: [X]
- Evidence citations from context: [X]
- Counterarguments addressed: [X]
- Critical weaknesses identified: [X]
- Data points supporting winner: [X]

**Debate Quality Metrics**:
- Argument diversity score: [X/10]
- Evidence reliability score: [X/10]
- Logical rigor score: [X/10]
- Overall debate quality: [X/10]

"""

# **Generation**

In [22]:
prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n\nAnswer the following question based on given context {context} \n\nQuestion: {question}\n\nYou are an expert debate moderator and analyst. Using only the information provided in the context, facilitate a structured debate on the topic presented in the question, then determine a clear winner with comprehensive statistics and visual representations.\n\n## Debate Structure:\n\n**Phase 1: Opening Statements**\nBased on the context provided, present 3-4 distinct perspectives on the topic. For each perspective:\n- State the core position clearly using evidence from the context\n- Provide 2-3 strongest supporting arguments found in the context\n- Include relevant statistics, data, or expert opinions from the provided context\n- Acknowledge the str

In [23]:
chain = prompt | llm

In [24]:
docs = retiever.invoke("who wins in a fight Gojo vs Sukuna")

In [25]:
debate_result = chain.invoke({"context":docs, "question":"who wins in a fight Gojo vs Sukuna"})

In [26]:
with open(r"debate_result.txt", "w") as f:
    f.write(debate_result)

In [28]:
rag_chain = {"context": retiever, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()

full_chain_result = rag_chain.invoke("is Gojo stronger than Sukuna")

In [29]:
full_chain_result

'The debate centers on the question: "Is Gojo stronger than Sukuna?" Based solely on the provided context, we will facilitate a structured debate to determine the answer.\n\n## Debate Structure:\n\n**Phase 1: Opening Statements**\n\n**Perspective 1: Gojo Satoru is unequivocally stronger.**\n\n*   **Core Position:** Gojo Satoru is explicitly declared the ultimate authority in terms of strength within this world, backed by an unparalleled defensive capability.\n*   **Strongest Supporting Arguments:**\n    1.  **Absolute Declaration of Strength:** The context directly states, "Gojo Satoru is the strongest." This is a general, overarching claim that places him at the pinnacle of power without qualification.\n    2.  **Superior Defensive Capabilities:** Gojo possesses an ability to "ignore the explosion" of a nuke, making him "hard to nuke." This demonstrates a level of invulnerability not attributed to any other character.\n*   **Relevant Statistics/Data from Context:** The direct quote "G

# **Query translation**